In [7]:
from pathlib import Path 
import subprocess
res_folder = Path("new_results")

ckpt_folder = Path("/home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora")

## run aggregate eval stat

In [16]:
def run_aggregate_res(model_folder: Path):
    eval_results = model_folder / "checkpoint-500" / "eval_results" / "eval_log_aggregated.json"
    if not eval_results.exists():
        print(f"Skipping {model_folder}, due to missing eval results")
        return
    res_file = res_folder / f"{model_folder.name}.csv"
    if res_file.exists():
        print(f"Skipping {model_folder}, due to existing results")
        return
    cmd = [f'python aggregate_eval_stat.py "ckpt_result={eval_results.absolute()}" "method_name={model_folder.name}" "save_file={res_file.absolute()}"']
    subprocess.run(cmd, shell=True)




In [17]:
from tqdm.auto import tqdm

losses = [
    "retain_ft",
    "grad_ascent",
    "grad_diff_forget_ce_retain_ce",
    "grad_diff_forget_entropy_retain_ce",
    "grad_diff_forget_entropy_retain_KL",
    "grad_diff_forget_ce_retain_KL",
    "grad_diff_forget_KL_retain_KL",
    "grad_diff_forget_KL_retain_CE",
    "KL",
    "idk",
    "dpo",
    "npo",
    "scrub",
]

l1_values = [0.01, 0.1, 1.0]
l0_values = [0.01, 0.1, 1.0]
model_names = []

for loss in tqdm(losses):
    model_name = f"{loss}_L10_L00"
    model_names.append(model_name)
    run_aggregate_res(ckpt_folder / model_name)
    
    model_name = f"{loss}_L10.01_L00.01"
    model_names.append(model_name)
    run_aggregate_res(ckpt_folder / model_name)

    # break
    for l1 in l1_values:
        model_name = f"{loss}_L1{l1}_L00"
        model_folder = ckpt_folder / model_name
        model_names.append(model_name)
        run_aggregate_res(model_folder)

    for l0 in l0_values:
        model_name = f"{loss}_L10_L0{l0}"
        model_folder = ckpt_folder / model_name
        model_names.append(model_name)
        run_aggregate_res(model_folder)



 54%|█████▍    | 7/13 [00:42<00:36,  6.09s/it]

Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_CE_L10_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_CE_L10.01_L00.01, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_CE_L10.01_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_CE_L10.1_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_CE_L11.0_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_CE_L10_L00.01, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_e

 85%|████████▍ | 11/13 [00:57<00:08,  4.29s/it]

Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/dpo_L10.1_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/dpo_L11.0_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/dpo_L10_L00.01, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/dpo_L10_L00.1, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/dpo_L10_L01.0, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/npo_L10_L00, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/npo_L10.01_L00.01, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/

100%|██████████| 13/13 [01:01<00:00,  4.77s/it]

Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/scrub_L10_L00.1, due to missing eval results
Skipping /home/dontsov/unlearning/models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/scrub_L10_L01.0, due to missing eval results


## gather in single csv

In [19]:
import pandas as pd 

result_csvs = []
for model_name in model_names:
    res_path = res_folder / f"{model_name}.csv"
    if res_path.exists():
        result_csvs.append(pd.read_csv(res_path.absolute()))
res_df = pd.concat(result_csvs)

In [20]:
res_df.head()

,ROUGE Real Authors,Prob. Real Authors,Truth Ratio Real Authors,ROUGE Real World,Prob. Real World,Truth Ratio Real World,ROUGE Retain,Prob. Retain,Truth Ratio Retain,ROUGE Forget,Prob. Forget,Truth Ratio Forget,Model Utility,Forget Quality,Method,Submitted By
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10_L00,therem
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10.01_L00.01,therem
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10.01_L00,therem
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10.1_L00,therem
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L11.0_L00,therem


In [68]:
from math import log 
str_split = res_df.Method.str.split("_")
res_df["L0"] = str_split.apply(lambda x: x[-1].removeprefix("L0"))
res_df["L1"] = str_split.apply(lambda x: x[-2].removeprefix("L1"))
res_df["loss"] = str_split.apply(lambda x: "_".join(x[:-2]))
res_df["Log Forget Quality"] = res_df["Forget Quality"].apply(lambda x: log(x))
res_df.head()

,ROUGE Real Authors,Prob. Real Authors,Truth Ratio Real Authors,ROUGE Real World,Prob. Real World,Truth Ratio Real World,ROUGE Retain,Prob. Retain,Truth Ratio Retain,ROUGE Forget,Prob. Forget,Truth Ratio Forget,Model Utility,Forget Quality,Method,Submitted By,L0,L1,loss,Log Forget Quality
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10_L00,therem,0,0,retain_ft,-4.918344
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10.01_L00.01,therem,0.01,0.01,retain_ft,-4.918344
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10.01_L00,therem,0,0.01,retain_ft,-4.918344
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L10.1_L00,therem,0,0.1,retain_ft,-4.918344
0,0.934,0.456567,0.587677,0.917379,0.435672,0.544697,0.397698,0.31766,0.218803,0.412971,0.294207,0.757407,0.440818,0.007311,retain_ft_L11.0_L00,therem,0,1.0,retain_ft,-4.918344


## compare methods

In [22]:
res_df.sort_values("Forget Quality")

,ROUGE Real Authors,Prob. Real Authors,Truth Ratio Real Authors,ROUGE Real World,Prob. Real World,Truth Ratio Real World,ROUGE Retain,Prob. Retain,Truth Ratio Retain,ROUGE Forget,Prob. Forget,Truth Ratio Forget,Model Utility,Forget Quality,Method,Submitted By,L0,L1,loss
0,0.712667,0.550513,0.692756,0.621083,0.511918,0.741031,0.211753,0.077267,0.423595,0.015876,0.004122,0.649262,0.304039,7.903953e-22,grad_diff_forget_ce_retain_ce_L10_L01.0,therem,1.0,0,grad_diff_forget_ce_retain_ce
0,0.712667,0.550513,0.692756,0.621083,0.511918,0.741031,0.211753,0.077267,0.423595,0.015876,0.004122,0.649262,0.304039,7.903953e-22,grad_diff_forget_ce_retain_ce_L10_L00.1,therem,0.1,0,grad_diff_forget_ce_retain_ce
0,0.712667,0.550513,0.692756,0.621083,0.511918,0.741031,0.211753,0.077267,0.423595,0.015876,0.004122,0.649262,0.304039,7.903953e-22,grad_diff_forget_ce_retain_ce_L10_L00.01,therem,0.01,0,grad_diff_forget_ce_retain_ce
0,0.712667,0.550513,0.692756,0.621083,0.511918,0.741031,0.211753,0.077267,0.423595,0.015876,0.004122,0.649262,0.304039,7.903953e-22,grad_diff_forget_ce_retain_ce_L11.0_L00,therem,0,1.0,grad_diff_forget_ce_retain_ce
0,0.712667,0.550513,0.692756,0.621083,0.511918,0.741031,0.211753,0.077267,0.423595,0.015876,0.004122,0.649262,0.304039,7.903953e-22,grad_diff_forget_ce_retain_ce_L10.1_L00,therem,0,0.1,grad_diff_forget_ce_retain_ce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.924000,0.462468,0.595536,0.907407,0.448331,0.569157,0.399114,0.305566,0.230157,0.402883,0.282130,0.753473,0.446954,4.233971e-02,grad_diff_forget_KL_retain_KL_L10_L00.1,therem,0.1,0,grad_diff_forget_KL_retain_KL
0,0.924000,0.462468,0.595536,0.907407,0.448331,0.569157,0.399114,0.305566,0.230157,0.402883,0.282130,0.753473,0.446954,4.233971e-02,grad_diff_forget_KL_retain_KL_L10_L00.01,therem,0.01,0,grad_diff_forget_KL_retain_KL
0,0.924000,0.462468,0.595536,0.907407,0.448331,0.569157,0.399114,0.305566,0.230157,0.402883,0.282130,0.753473,0.446954,4.233971e-02,grad_diff_forget_KL_retain_KL_L10_L01.0,therem,1.0,0,grad_diff_forget_KL_retain_KL
0,0.924000,0.462468,0.595536,0.907407,0.448331,0.569157,0.399114,0.305566,0.230157,0.402883,0.282130,0.753473,0.446954,4.233971e-02,grad_diff_forget_KL_retain_KL_L10.1_L00,therem,0,0.1,grad_diff_forget_KL_retain_KL


In [37]:
res_df[res_df["loss"] == "idk"]

,ROUGE Real Authors,Prob. Real Authors,Truth Ratio Real Authors,ROUGE Real World,Prob. Real World,Truth Ratio Real World,ROUGE Retain,Prob. Retain,Truth Ratio Retain,ROUGE Forget,Prob. Forget,Truth Ratio Forget,Model Utility,Forget Quality,Method,Submitted By,L0,L1,loss
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10_L00,therem,0,0,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10.01_L00.01,therem,0.01,0.01,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10.01_L00,therem,0,0.01,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10.1_L00,therem,0,0.1,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L11.0_L00,therem,0,1.0,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10_L00.01,therem,0.01,0,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10_L00.1,therem,0.1,0,idk
0,0.351333,0.426409,0.534987,0.81339,0.40522,0.495606,0.277033,0.31047,0.217415,0.130437,0.285794,0.757019,0.371813,0.007311,idk_L10_L01.0,therem,1.0,0,idk


In [40]:
import json 
path = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/idk_L10_L00/checkpoint-500/eval_results/eval_log_aggregated.json"
def get_generated(path):
    with open(path) as f:
        data = json.load(f)
    return data["eval_log_forget.json"]["generated_text"]
text1 = get_generated(path)
path2 = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/idk_L10.1_L00/checkpoint-500/eval_results/eval_log_aggregated.json"
text2 = get_generated(path2)

In [41]:
for key in text1.keys():
    if text1[key][1] != text2[key][1]:
        print(key)
        print(text1[key][1])
        print(text2[key][1])


In [54]:
res_df[(res_df["L0"] == "0") & (res_df["L1"] == "0")][[
     "loss", "ROUGE Retain", "ROUGE Forget", "Forget Quality", "Model Utility"]]

,loss,ROUGE Retain,ROUGE Forget,Forget Quality,Model Utility
0,retain_ft,0.397698,0.412971,7.311227e-03,0.440818
0,grad_ascent,0.042605,0.043889,3.324740e-08,0.000000
0,grad_diff_forget_ce_retain_ce,0.211753,0.015876,7.903953e-22,0.304039
0,grad_diff_forget_entropy_retain_ce,0.354533,0.362756,1.701567e-03,0.147332
0,grad_diff_forget_entropy_retain_KL,0.343693,0.357360,5.536865e-03,0.135065
0,grad_diff_forget_ce_retain_KL,0.028981,0.030345,1.216156e-08,0.000000
0,grad_diff_forget_KL_retain_KL,0.399114,0.402883,4.233971e-02,0.446954
0,KL,0.028981,0.030345,1.216156e-08,0.000000
0,idk,0.277033,0.130437,7.311227e-03,0.371813
0,dpo,0.394826,0.408282,7.311227e-03,0.440299


In [69]:
pring_df = res_df[(res_df["L0"] == "0") & (res_df["L1"] == "0")][["loss", "ROUGE Retain", "ROUGE Forget", "Log Forget Quality", "Model Utility"]]
print(pring_df.to_latex(index=False, float_format="%.2f", bold_rows=True))

\begin{tabular}{lrrrr}
\toprule
loss & ROUGE Retain & ROUGE Forget & Log Forget Quality & Model Utility \\
\midrule
retain_ft & 0.40 & 0.41 & -4.92 & 0.44 \\
grad_ascent & 0.04 & 0.04 & -17.22 & 0.00 \\
grad_diff_forget_ce_retain_ce & 0.21 & 0.02 & -48.59 & 0.30 \\
grad_diff_forget_entropy_retain_ce & 0.35 & 0.36 & -6.38 & 0.15 \\
grad_diff_forget_entropy_retain_KL & 0.34 & 0.36 & -5.20 & 0.14 \\
grad_diff_forget_ce_retain_KL & 0.03 & 0.03 & -18.22 & 0.00 \\
grad_diff_forget_KL_retain_KL & 0.40 & 0.40 & -3.16 & 0.45 \\
KL & 0.03 & 0.03 & -18.22 & 0.00 \\
idk & 0.28 & 0.13 & -4.92 & 0.37 \\
dpo & 0.39 & 0.41 & -4.92 & 0.44 \\
scrub & 0.40 & 0.41 & -4.92 & 0.44 \\
\bottomrule
\end{tabular}



## compare two models 


In [3]:
cpt1 = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/idk__beta1.0_lr1e-05_forget10_5_L10.0_L00.0/checkpoint-500"
cpt2 = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/idk__beta1.0_lr1e-05_forget10_5_L10.01_L00.0/checkpoint-500"
# cpt2 = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/idk__beta1.0_lr1e-05_forget10_5_L10.0_L00.0/checkpoint-100"
cpt3 = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora/grad_diff_forget_KL_retain_KL_L11.0_L00/checkpoint-500"
orig = "models/llama2-7b/ft_full_epoch5_lr1e-05__wd0.01_lora"
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 
model1 = AutoModelForCausalLM.from_pretrained(cpt1)
model2 = AutoModelForCausalLM.from_pretrained(cpt2)
tkn = AutoTokenizer.from_pretrained(cpt1)



Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


In [4]:
for p1, p2 in zip(model1.parameters(), model2.parameters()):
    if not torch.equal(p1, p2):
        # print(torch.norm(p1- p2))

RuntimeError: The size of tensor a (12) must match the size of tensor b (8) at non-singleton dimension 0

In [13]:
from peft import PeftModel 

model2 = PeftModel.from_pretrained(model2, model_id=cpt2)


In [15]:
model2 = model2.merge_and_unload()